# Specs

In [1]:
!nvidia-smi

Sun Mar 12 19:04:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   26C    P0    49W / 400W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Imports

In [2]:
import os, gc
import copy
import math
import random
import getpass
import pandas as pd
import numpy as np
from time import time, sleep
from tqdm.notebook import tqdm
from collections import Counter, defaultdict
from scipy.ndimage import gaussian_filter

import shutil

import cv2
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import f1_score, mean_squared_error

In [3]:
from glob import glob

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset as TDataset, DataLoader
from torch.optim.lr_scheduler import LambdaLR, CosineAnnealingLR

from accelerate import Accelerator

import albumentations as A
import timm

In [5]:
%load_ext watermark
%watermark --iversions

pandas        : 1.5.0
albumentations: 1.3.0
matplotlib    : 3.6.1
cv2           : 4.7.0
numpy         : 1.23.4
torch         : 1.13.1+cu116
timm          : 0.6.12



# Envs

In [6]:
SEED = 42

import warnings
warnings.filterwarnings('ignore')

# Data

In [7]:
# load files
data_path = './data/'

train = pd.read_csv(data_path + 'Train.csv')
test = pd.read_csv(data_path + 'Test.csv')
sample_submission = pd.read_csv(data_path + 'SampleSubmission.csv')

# Preview train
train.head()

,ImageId,Target
0,Id_jdqw9hlv6j.png,14.0
1,Id_6xtrolmuvc.png,18.0
2,Id_2m49sj3xd9.png,0.0
3,Id_9jwg5pcnn4.png,28.0
4,Id_vnm6e8n0p3.png,21.0


In [8]:
# Preview test
test.head()

,ImageId
0,Id_ohk78h9ld8.png
1,Id_eeyj2u4j7y.png
2,Id_wsd7vx2ifa.png
3,Id_6vfneamaoh.png
4,Id_9wil3575fv.png


In [9]:
# Preview sample submission
sample_submission.head()

,ImageId,Target
0,Id_ohk78h9ld8.png,0
1,Id_eeyj2u4j7y.png,0
2,Id_wsd7vx2ifa.png,0
3,Id_6vfneamaoh.png,0
4,Id_9wil3575fv.png,0


# Utils

In [10]:
def get_common_transforms(size):
    return A.Compose([
      A.Resize(height=size, width=size, p=1),
    ])

In [11]:
def criterion(preds, biomass):
    loss_fn = nn.MSELoss()
    return torch.sqrt(loss_fn(preds, biomass))

In [12]:
class TreeCountingModel(nn.Module):
    def __init__(self, args):
        super().__init__()

        self.backbone = timm.create_model(
            args.arch, num_classes=1, pretrained=False, **args.extra_params
        )

    def forward(self, x):
        x = self.backbone(x)
        return x

In [13]:
class TreeCountingDataset(torch.utils.data.Dataset):
    def __init__(self, args, imagesPath, df, tfms=None):
        self.imagespath = imagesPath
        self.df = df.reset_index(drop=True)
        self.tfms = tfms
        self.default_tfms = get_common_transforms(args.img_size)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        imagename, label, *z = self.df.iloc[idx].values
        
        path = os.path.join(self.imagespath, imagename)
        image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        
        if self.tfms is not None:
            image = self.tfms(image=image)['image']
        
        image = self.default_tfms(image=image)['image']
        image = torch.tensor(image, dtype=torch.float)
        image = torch.permute(image, (2, 0, 1))
        
        return image, torch.tensor(label, dtype=torch.float)

In [14]:
def get_model(args):
    model = TreeCountingModel(args)

    return model

def load_model(args, fold, model_path, mm='best'):
    model = get_model(args)
    model.load_state_dict(torch.load(f'{model_path}/{mm}_{fold}.pt'))
    model.eval()
    
    return model

In [15]:
def load_splits(args):   
    all_splits = {}
    for f in range(args.n_split):
        all_splits[f] = {
          'val': pd.read_csv(f"folds/{args.n_split}_splits/fold_{f}.csv")
        }
        
    return all_splits

In [16]:
def infer(dataloader, model):
    biomasses = []

    model.eval()
    with torch.no_grad():
        for i, (x, biomass) in enumerate(tqdm(dataloader)):
            preds = model(x).squeeze(dim=-1).detach().cpu().numpy()
            preds = np.where(preds > 0, preds, 0)

            biomasses.extend(preds)

    return biomasses


################## Run training over folds Function ####################   
def release_memory():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.synchronize()

def run_fold(fold, args, path='./models'):
    model_path = f'{path}/{args.arch}-{args.n_split}-{args.img_size}-{args.lr}-{args.bs}'
    model_path = sorted(glob(f'{model_path}*'))[-1]
    
    if fold == 0:
        print('Using model located at:', model_path)

    test_ds = TreeCountingDataset(
        args,
        imagesPath='./data/TreeImages/',
        df=sample_submission,
    )

    test_dl = DataLoader(test_ds, batch_size=args.ebs, num_workers=args.workers, pin_memory=True, shuffle=False)

    accelerator = Accelerator(fp16=args.fp16)

    model = load_model(args, fold, model_path)
    model, test_dl = accelerator.prepare(model, test_dl)

    preds = infer(test_dl, model)

    release_memory()

    return preds

def run_inference(args):
    all_preds = [
        run_fold(fold, args) for fold in args.use_folds
    ]
    
    return np.mean(all_preds, axis=0)

In [17]:
class InferencePipeline:
    def __init__(self, cfg):
        self.accelerator = Accelerator(fp16=cfg.fp16)
        self.cfg = cfg
    
    def _predict_fn(self, dataloader, model):
        palm_count = []
        
        model.eval()
        with torch.no_grad():
            for i, (x, biomass) in enumerate(tqdm(dataloader)):
                preds = model(x).squeeze(dim=-1).detach().cpu().numpy()
                preds = np.where(preds > 0, preds, 0)

                palm_count.extend(preds)

        return palm_count

    def run_prediction(self, path = './models'):
        all_preds = []
        
        model_path = f'{path}/{self.cfg.arch}-{self.cfg.n_split}-{self.cfg.img_size}-{self.cfg.lr}-{self.cfg.bs}-{self.cfg.model_id}'    
        print('Using model located at:', model_path)
    
        for fold in tqdm(self.cfg.use_folds):
            fold_preds = []
            
            model = load_model(self.cfg, fold, model_path)
            model = self.accelerator.prepare(model)
        
            # Simple Prediction
            ds = TreeCountingDataset(self.cfg, imagesPath='./data/TreeImages/', df=sample_submission)
            dl = DataLoader(ds, self.cfg.ebs, shuffle=False, num_workers=self.cfg.workers)
            dl = self.accelerator.prepare(dl)
            fold_preds.append(
                self._predict_fn(dl, model)
            )

            # TTA
            for n, tfm in enumerate(self.cfg.tta_tfms):
                ds = TreeCountingDataset(self.cfg, imagesPath='./data/TreeImages/', df=sample_submission, tfms=tfm)
                dl = DataLoader(ds, self.cfg.ebs, shuffle=False, num_workers=self.cfg.workers)
                dl = self.accelerator.prepare(dl)
                fold_preds.append(
                    self._predict_fn(dl, model)
                )
            
            release_memory()

            all_preds.extend(fold_preds)

        return all_preds

In [18]:
class ValidationPipeline:
    def __init__(self, cfg):
        self.cfg = cfg
        self.accelerator = Accelerator(fp16=cfg.fp16)
    
    def _predict_fn(self, dataloader, model):    
        all_logits = []
        all_labels = []

        model.eval()
        with torch.no_grad():
            for i, (x, label) in enumerate(dataloader):
                preds = model(x).squeeze(dim=-1).detach().cpu().numpy()
                preds = np.where(preds > 0, preds, 0)

                all_logits.extend(preds)
                all_labels.extend(label.detach().cpu().numpy())

        return all_logits, all_labels
    
    def run_evaluation(self, path = './models'):
        self.all_val_preds, self.all_val_labels = [], []
        
        model_path = f'{path}/{self.cfg.arch}-{self.cfg.n_split}-{self.cfg.img_size}-{self.cfg.lr}-{self.cfg.bs}-{self.cfg.model_id}'    
        print('Using model located at:', model_path)

        accelerator = Accelerator(fp16=self.cfg.fp16)
    
        for fold in tqdm(self.cfg.use_folds, desc='Evaluation'):
            model = load_model(self.cfg, fold, model_path)
            model = self.accelerator.prepare(model)
            
            ds = TreeCountingDataset(self.cfg, imagesPath='./data/TreeImages/', df=all_splits[fold]['val'])
            dl = DataLoader(ds, self.cfg.ebs, shuffle=False, num_workers=self.cfg.workers)
            dl = self.accelerator.prepare(dl)
        
            v_preds, v_labels = self._predict_fn(dl, model)

            self.all_val_preds.extend(v_preds)
            self.all_val_labels.extend(v_labels)
            
            release_memory()
            
        self.all_val_labels = np.array(self.all_val_labels)
        self.all_val_preds = np.array(self.all_val_preds)
        
        return self.all_val_labels, self.all_val_preds
            
    def run_evaluation_tta(self, path = './models'):
        self.all_val_preds_tta, self.all_val_labels_tta = [], []
        
        model_path = f'{path}/{self.cfg.arch}-{self.cfg.n_split}-{self.cfg.img_size}-{self.cfg.lr}-{self.cfg.bs}-{self.cfg.model_id}'    
        print('Using model located at:', model_path)

        accelerator = Accelerator(fp16=self.cfg.fp16)
    
        for fold in tqdm(self.cfg.use_folds, desc='Evaluation'):
            fold_preds = []
            
            model = load_model(self.cfg, fold, model_path)
            model = self.accelerator.prepare(model)
            
            ds = TreeCountingDataset(self.cfg, imagesPath='./data/TreeImages/', df=all_splits[fold]['val'])
            dl = DataLoader(ds, self.cfg.ebs, shuffle=False, num_workers=self.cfg.workers)
            dl = self.accelerator.prepare(dl)
        
            v_preds, v_labels = self._predict_fn(dl, model)
            fold_preds.append(v_preds)
            
            for n, tfm in enumerate(self.cfg.tta_tfms):
                ds = TreeCountingDataset(self.cfg, imagesPath='./data/TreeImages/', df=all_splits[fold]['val'], tfms=tfm)
                dl = DataLoader(ds, self.cfg.ebs, shuffle=False, num_workers=self.cfg.workers)
                dl = self.accelerator.prepare(dl)
                v_preds, _ = self._predict_fn(dl, model)
                
                fold_preds.append(v_preds)

            self.all_val_preds_tta.extend(np.mean(fold_preds, axis=0))
            self.all_val_labels_tta.extend(v_labels)
            
            release_memory()
            
        self.all_val_labels = np.array(self.all_val_labels)
        self.all_val_preds = np.array(self.all_val_preds)
        
        return self.all_val_labels, self.all_val_preds
    
    @property
    def score(self):
        return mean_squared_error(self.all_val_labels, self.all_val_preds, squared=False)
    
    @property
    def score_tta(self):
        return mean_squared_error(self.all_val_labels_tta, self.all_val_preds_tta, squared=False)

# Training

In [19]:
class CFG:
    arch = 'tf_efficientnetv2_s_in21ft1k'
    lr = 5e-4
    min_lr = 0
    wd = 1e-6
    epochs = 20
    warmup = 0.
    bs = 24
    ebs = 2 * bs
    
    n_split = 5
    use_folds = list(range(n_split))
    actual_split = len(use_folds)

    img_size = 1024
    model_id = 0
    
    tta_tfms = []

    schedule = True
    augment = True
    mosaic = False
    mosaic_aug = False

    fp16 = True
    workers = os.cpu_count()
    
    print('Cpu count:', workers)

    extra_params = {
    }
    
    def __init__(self, **kwargs):
        self.__update(**kwargs)
    
    def __update(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)

Cpu count: 12


In [20]:
all_splits = load_splits(CFG())

In [21]:
tta_tfms = [
    A.HorizontalFlip(p=1.0),       
    A.VerticalFlip(p=1.0),
    A.Blur(blur_limit=3, p=1.0),
    A.Compose([
        A.HorizontalFlip(p=1.0),       
        A.VerticalFlip(p=1.0),
        A.Blur(blur_limit=3, p=1.0),
    ])
]

In [22]:
regnet_args = CFG(arch = 'regnetz_040h', lr = 1e-3, img_size=768, bs=24, model_id=0, tta_tfms=tta_tfms)
seresnet_args = CFG(arch = 'seresnet152d', lr = 5e-4, img_size=1024, bs=24, model_id=0, tta_tfms=tta_tfms)
effv2s_args = CFG(arch = 'tf_efficientnetv2_s_in21ft1k', lr = 5e-4, img_size=1024, bs=24, model_id=0, tta_tfms=tta_tfms)
convn_args = CFG(arch = 'convnext_small_384_in22ft1k', lr = 1e-4, img_size=1024, bs=8, model_id=0, tta_tfms=tta_tfms)

In [23]:
val_preds = []
val_labels = []

for args in tqdm([regnet_args, seresnet_args, effv2s_args, convn_args]):
    
    val_pipeline = ValidationPipeline(args)
    _ = val_pipeline.run_evaluation()
    print('Validation score:', val_pipeline.score)
    _ = val_pipeline.run_evaluation_tta() 
    print('TTA Validation score:', val_pipeline.score_tta)

    val_preds.append(
        val_pipeline.all_val_preds_tta
    )
    val_labels = val_pipeline.all_val_labels

  0%|          | 0/4 [00:00<?, ?it/s]

Using model located at: ./models/regnetz_040h-5-768-0.001-24-1


Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Validation score: 1.377299
Using model located at: ./models/regnetz_040h-5-768-0.001-24-1


Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

...

TTA Validation score: 1.3813378
Using model located at: ./models/convnext_small_384_in22ft1k-5-1024-0.0001-8-0


Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Validation score: 1.4548277
Using model located at: ./models/convnext_small_384_in22ft1k-5-1024-0.0001-8-0


Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

TTA Validation score: 1.4438583


In [24]:
from itertools import combinations, combinations_with_replacement, permutations

In [28]:
n = 4
w = np.arange(0, 1, 0.005)
combs = list(combinations_with_replacement(w, n))
combs = [list(x) for x in combs if sum(x)==1]

In [29]:
best_score = np.inf
best_w = None
best_blend = None

for comb in tqdm(combs):
    for p_w in permutations(comb):
        blend = np.average(val_preds[:n], weights=p_w, axis=0)
        sc = mean_squared_error(val_labels, blend, squared=False)
        
        if sc < best_score:
            best_score = sc
            best_w = p_w
            best_blend = blend

  0%|          | 0/59663 [00:00<?, ?it/s]

In [30]:
best_score, best_w

(1.256211318897102, (0.34500000000000003, 0.24, 0.155, 0.26))

In [ ]:
best_w = (0.345, 0.24, 0.155, 0.26)

In [47]:
blend_preds = []

for args in tqdm([regnet_args, seresnet_args, effv2s_args, convn_args]):
    
    pipeline = InferencePipeline(args)
    blend_preds.append(
        np.mean(pipeline.run_prediction(), axis=0)
    )
    
preds = np.average(blend_preds, weights=best_w, axis=0)

  0%|          | 0/4 [00:00<?, ?it/s]

Using model located at: ./models/regnetz_040h-5-768-0.001-24-1


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

...

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [48]:
sample_submission['Target'] = preds

In [49]:
sample_submission.head()

,ImageId,Target
0,Id_ohk78h9ld8.png,0.016857
1,Id_eeyj2u4j7y.png,0.032991
2,Id_wsd7vx2ifa.png,13.717153
3,Id_6vfneamaoh.png,12.933681
4,Id_9wil3575fv.png,14.621752


In [50]:
sample_submission.describe()

,Target
count,858.000000
mean,9.959156
std,13.441103
min,0.008500
25%,0.016857
50%,1.585274
75%,16.200991
max,45.648655


In [51]:
!mkdir submissions

mkdir: cannot create directory ‘submissions’: File exists
mkdir: cannot create directory ‘submissions/blend’: File exists


In [52]:
sample_submission.to_csv(
    f'submissions/submission_wmm.csv', 
    index=False
)